# Who is the best pokemon?

importing necessary libraries

In [19]:
from determine_job import DetermineJob
from generate_data import generate_random_pokemon
from simulate import EloAlgo
import pandas as pd
import numpy as np
import requests
import os
import random

import warnings
warnings.filterwarnings("ignore")

### Getting the data from API

In [20]:
#parse_json takes a string url as input and then returns a dict with info of the specific pokemon from the API

def parse_json(url, isInfo):
    response = requests.get(url)
    content = response.json()
    
    if isInfo:
        name = content['name']
        moveset = ', '.join([move['move']['name'] for move in content['moves']])
        types = ', '.join([p_type['type']['name'] for p_type in content['types']])
        stats = [stat['base_stat'] for stat in content['stats']]
        
        poke_info = {'NAME': name , 'TYPE': types, 'HP': stats[0], 'ATTACK': stats[1], 'DEFENSE': stats[2], 
                                    'SPECIAL-ATTACK': stats[3], 'SPECIAL-DEFENSE': stats[4], 'SPEED': stats[5], 
                                    'TOTAL': sum(stats), 'MOVESET': moveset}
    else:
        p_type_name = content['name']
        poke_info = {p_type_name: {relation: [name['name'] for name in content['damage_relations'][relation]] 
                                   for relation in content['damage_relations']}}
        

    return poke_info
        

### Storing Pokemon Info in Excel File

Run through every pokemon in the pokeDB API and then store them in a Dataframe called poke_df

Done through this method to lessen time to debug and not constantly deal with timeout on API

In [21]:
if not os.path.isfile('excel_files/poke_db.xlsx'):
    poke_df = pd.DataFrame(columns=['NAME', 'TYPE', 'HP', 'ATTACK', 'DEFENSE', 
                                'SPECIAL-ATTACK', 'SPECIAL-DEFENSE', 'SPEED', 
                                'TOTAL', 'MOVESET'])

    poke_id = 1
    poke_id_secondary = 10001

    while True:
        try:
            pID = poke_id_secondary if poke_id > 1025 else poke_id
            poke_df = pd.concat([poke_df, pd.DataFrame([parse_json(f'https://pokeapi.co/api/v2/pokemon/{pID}/', True)])], ignore_index=True)

            if poke_id > 1025:
                poke_id_secondary += 1
            else:
                poke_id += 1
        except:
            break
        
    poke_df.to_excel('excel_files/poke_db.xlsx')
else:
    poke_df = pd.read_excel('excel_files/poke_db.xlsx')
    
poke_df = poke_df.drop('Unnamed: 0', axis=1)
poke_df.head()

,NAME,TYPE,HP,ATTACK,DEFENSE,SPECIAL-ATTACK,SPECIAL-DEFENSE,SPEED,TOTAL,MOVESET
0,bulbasaur,"grass, poison",45,49,49,65,65,45,318,"razor-wind, swords-dance, cut, bind, vine-whip..."
1,ivysaur,"grass, poison",60,62,63,80,80,60,405,"swords-dance, cut, bind, vine-whip, headbutt, ..."
2,venusaur,"grass, poison",80,82,83,100,100,80,525,"swords-dance, cut, bind, vine-whip, headbutt, ..."
3,charmander,fire,39,52,43,60,50,65,309,"mega-punch, fire-punch, thunder-punch, scratch..."
4,charmeleon,fire,58,64,58,80,65,80,405,"mega-punch, fire-punch, thunder-punch, scratch..."


### Storing Type Advantages in Excel File

Get the type advantages and input them into a dictionary and excel file to handle simulation of matches

In [22]:
if not os.path.isfile('excel_files/type_advantages.xlsx'):
    type_id = 1
    types = dict()
    
    while True:
        try:
            types.update(parse_json(f"https://pokeapi.co/api/v2/type/{type_id}", False))
            type_id += 1
        except:
            break
        
    move_adv_df = pd.DataFrame.from_dict(types)
    move_adv_df.to_excel('excel_files/type_advantages.xlsx')
else:
    move_adv_df = pd.read_excel('excel_files/type_advantages.xlsx')
    
move_adv_df = move_adv_df.drop('Unnamed: 0', axis=1)
move_adv_df.head()

,normal,fighting,flying,poison,ground,rock,bug,ghost,steel,fire,water,grass,electric,psychic,ice,dragon,dark,fairy
0,['fighting'],"['flying', 'psychic', 'fairy']","['rock', 'electric', 'ice']","['ground', 'psychic']","['water', 'grass', 'ice']","['fighting', 'ground', 'steel', 'water', 'grass']","['flying', 'rock', 'fire']","['ghost', 'dark']","['fighting', 'ground', 'fire']","['ground', 'rock', 'water']","['grass', 'electric']","['flying', 'poison', 'bug', 'fire', 'ice']",['ground'],"['bug', 'ghost', 'dark']","['fighting', 'rock', 'steel', 'fire']","['ice', 'dragon', 'fairy']","['fighting', 'bug', 'fairy']","['poison', 'steel']"
1,[],"['normal', 'rock', 'steel', 'ice', 'dark']","['fighting', 'bug', 'grass']","['grass', 'fairy']","['poison', 'rock', 'steel', 'fire', 'electric']","['flying', 'bug', 'fire', 'ice']","['grass', 'psychic', 'dark']","['ghost', 'psychic']","['rock', 'ice', 'fairy']","['bug', 'steel', 'grass', 'ice']","['ground', 'rock', 'fire']","['ground', 'rock', 'water']","['flying', 'water']","['fighting', 'poison']","['flying', 'ground', 'grass', 'dragon']",['dragon'],"['ghost', 'psychic']","['fighting', 'dragon', 'dark']"
2,[],"['rock', 'bug', 'dark']","['fighting', 'bug', 'grass']","['fighting', 'poison', 'bug', 'grass', 'fairy']","['poison', 'rock']","['normal', 'flying', 'poison', 'fire']","['fighting', 'ground', 'grass']","['poison', 'bug']","['normal', 'flying', 'rock', 'bug', 'steel', '...","['bug', 'steel', 'fire', 'grass', 'ice', 'fairy']","['steel', 'fire', 'water', 'ice']","['ground', 'water', 'grass', 'electric']","['flying', 'steel', 'electric']","['fighting', 'psychic']",['ice'],"['fire', 'water', 'grass', 'electric']","['ghost', 'dark']","['fighting', 'bug', 'dark']"
3,"['rock', 'steel']","['flying', 'poison', 'bug', 'psychic', 'fairy']","['rock', 'steel', 'electric']","['poison', 'ground', 'rock', 'ghost']","['bug', 'grass']","['fighting', 'ground', 'steel']","['fighting', 'flying', 'poison', 'ghost', 'ste...",['dark'],"['steel', 'fire', 'water', 'electric']","['rock', 'fire', 'water', 'dragon']","['water', 'grass', 'dragon']","['flying', 'poison', 'bug', 'steel', 'fire', '...","['grass', 'electric', 'dragon']","['steel', 'psychic']","['steel', 'fire', 'water', 'ice']",['steel'],"['fighting', 'dark', 'fairy']","['poison', 'steel', 'fire']"
4,['ghost'],[],['ground'],[],['electric'],[],[],"['normal', 'fighting']",['poison'],[],[],[],[],[],[],[],['psychic'],['dragon']


### Finding Stat Spread of all Pokemons

The point of this is that the way that when jobs will need to be determined later, we can use the average numbers for the base stats for most pokemon where their job does not require a specific upper bound stat

Ex: For a pokemon that is a physical sweeper, by standard pokemon users, the physical sweeper must have minimum 101 speed and 110 attack to be considered strong, the rest of the stats are irrelevant thus they can be the base stats

In [23]:
stat_names = ['HP', 'ATTACK', 'DEFENSE', 'SPECIAL-ATTACK', 'SPECIAL-DEFENSE', 'SPEED']

for name in stat_names:
    print(f"Average {name} is {round(np.mean(poke_df[name]))}")
    

Average HP is 71
Average ATTACK is 82
Average DEFENSE is 75
Average SPECIAL-ATTACK is 74
Average SPECIAL-DEFENSE is 73
Average SPEED is 71


### Testing accuracy of DetermineJob Class

The DetermineJob class has multiple methods, the method that is used is get_jobs_primary and get_jobs_secondary, 
they take input of an array of stats and their moveset, they outputs an array of their jobs

This is simply to test how accurate the job assigning system is, from the 100 different pokemons, 
it seems almost perfectly accurate in what their roles are

In [24]:
job_determine = DetermineJob()

for i in range(100):
    print(poke_df.iloc[i]['NAME'], job_determine.get_jobs_primary([poke_df.iloc[i]['HP'], poke_df.iloc[i]['ATTACK'], poke_df.iloc[i]['DEFENSE'], 
                           poke_df.iloc[i]['SPECIAL-ATTACK'], poke_df.iloc[i]['SPECIAL-DEFENSE'], poke_df.iloc[i]['SPEED']]))
    print(job_determine.get_jobs_secondary(poke_df.iloc[i]['MOVESET']))

bulbasaur Jack-Of-All-Trades
['Staller', 'Toxi-Shufflers']
ivysaur Jack-Of-All-Trades
['Staller', 'Toxi-Shufflers']
venusaur Jack-Of-All-Trades
['Pseudo-Hazer', 'Staller', 'Toxi-Shufflers', 'Para-Shufflers']
charmander Special Sweeper
['Pseudo-Hazer', 'Staller', 'Toxi-Shufflers', 'Para-Shufflers']
charmeleon Special Sweeper
['Pseudo-Hazer', 'Staller', 'Toxi-Shufflers', 'Para-Shufflers']
charizard Special Sweeper
['Pseudo-Hazer', 'Staller', 'Toxi-Shufflers', 'Para-Shufflers']
squirtle Wall
['Hazer', 'Staller', 'Toxi-Shufflers']
wartortle Wall
['Staller', 'Toxi-Shufflers']
blastoise Jack-Of-All-Trades
['Pseudo-Hazer', 'Staller', 'Toxi-Shufflers', 'Para-Shufflers']
caterpie Jack-Of-All-Trades
[]
metapod Physical Tank
[]
butterfree Jack-Of-All-Trades
['Pseudo-Hazer', 'Staller', 'Toxi-Shufflers', 'Para-Shufflers', 'Pivots']
weedle Physical Sweeper
[]
kakuna Physical Tank
[]
beedrill Physical Sweeper
['Staller', 'Toxi-Shufflers', 'Pivots']
pidgey Physical Sweeper
['Pseudo-Hazer', 'Staller', 

### Creating Synthetic Data to predict actual Pokemons

Creating a dataset consisting of 10,000 random pokemons which will be assinged jobs, once they are assinged jobs, the dataset will be used to predict the jobs of the actual pokemon

The idea of this is to basically create comepletely random pokemon with random stats and based off of this, they will be assinged a job using DetermineJob's get_jobs_primary method, this method is used since it is based only on stats whereas the get_jobs_secondary is only off of their moveset which doesn't require any predictions

In [25]:
pokemon_df = generate_random_pokemon(100000)
pokemon_df.sample(n=10)

,HP,Attack,Defense,Special-Attack,Special-Defense,Speed,Job
88644,196,161,56,55,96,172,Jack-Of-All-Trades
16036,208,68,203,119,183,167,Wall
50912,46,14,113,162,52,143,Special Sweeper
23156,56,86,214,57,185,91,Wall
48310,140,14,189,51,112,49,Wall
29776,18,103,221,90,61,182,Jack-Of-All-Trades
65928,171,79,210,91,103,86,Physical Tank
42901,59,115,110,187,72,10,Special Sweeper
63677,157,19,98,125,126,180,Jack-Of-All-Trades
53335,108,126,132,34,143,92,Wall


### Creating Model for Predictions

Using the pokemon_df dataset created to predict the jobs of actual pokemons

RandomForest was used because jobs are more cluster based rather than being linear based, they can vary a lot in terms of their stats spread

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

pokemon_df['Job'] = pokemon_df['Job'].astype('category')

X_train, X_test, y_train, y_test = train_test_split(pokemon_df.drop(['Job'], axis='columns'), pokemon_df.Job, test_size=0.2, random_state=42)
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9231

### Using Model to Predict Pokemon Jobs

Assinging the primary job to the pokemons, this is only stats based

Also assinging secondary jobs which is based on movesets

In [27]:
poke_df['Primary Job'.upper()] = None
poke_df['Secondary Job'.upper()] = None

for index, row in poke_df.iterrows():
    stats = [poke_df.at[index, stat.upper()] for stat in stat_names]
    moveset = poke_df.at[index, 'MOVESET']
    if type(moveset) != float:
        poke_df.at[index, 'Secondary Job'.upper()] = (', ').join(job_determine.get_jobs_secondary(moveset)).upper()
    poke_df.at[index, 'Primary Job'.upper()] = clf.predict([stats])
    
poke_df.sample(n=10)

,NAME,TYPE,HP,ATTACK,DEFENSE,SPECIAL-ATTACK,SPECIAL-DEFENSE,SPEED,TOTAL,MOVESET,PRIMARY JOB,SECONDARY JOB
447,lucario,"fighting, steel",70,110,70,115,70,90,525,"mega-punch, ice-punch, thunder-punch, swords-d...",Mixed Sweeper,"PSEUDO-HAZER, STALLER, TOXI-SHUFFLERS, PARA-SH..."
883,duraludon,"steel, dragon",70,95,115,120,50,85,535,"swords-dance, body-slam, leer, hyper-beam, sol...",Jack-Of-All-Trades,STALLER
413,mothim,"bug, flying",70,94,50,94,50,66,424,"gust, tackle, psybeam, hyper-beam, solar-beam,...",Mixed Sweeper,"STALLER, TOXI-SHUFFLERS, PIVOTS"
300,delcatty,normal,70,65,65,55,55,90,400,"double-slap, headbutt, tackle, body-slam, doub...",Jack-Of-All-Trades,"STALLER, TOXI-SHUFFLERS"
597,ferrothorn,"grass, steel",74,94,131,54,116,20,489,"swords-dance, cut, tackle, pin-missile, hyper-...",Wall,"STALLER, SPIKER, TOXI-SHUFFLERS"
196,umbreon,dark,95,65,110,60,130,65,525,"pay-day, cut, sand-attack, headbutt, tackle, b...",Wall,"PSEUDO-HAZER, STALLER, TOXI-SHUFFLERS, PARA-SH..."
273,nuzleaf,"grass, dark",70,70,40,60,40,60,340,"pound, razor-wind, swords-dance, cut, mega-kic...",Jack-Of-All-Trades,"STALLER, TOXI-SHUFFLERS"
738,crabrawler,fighting,47,82,57,42,47,63,338,"ice-punch, thunder-punch, slam, body-slam, tak...",Physical Sweeper,"STALLER, TOXI-SHUFFLERS"
864,sirfetchd,fighting,62,135,95,68,82,65,507,"swords-dance, slam, sand-attack, body-slam, le...",Physical Sweeper,STALLER
0,bulbasaur,"grass, poison",45,49,49,65,65,45,318,"razor-wind, swords-dance, cut, bind, vine-whip...",Jack-Of-All-Trades,"STALLER, TOXI-SHUFFLERS"


### Elo Algorithm on Pokemons

Pokemon's starting ELO is based off their total base stats as this is a general metric to understand how good pokemons are


In [28]:
simulator = EloAlgo()

def handle_elo(df, newcol_name, from_col):
    df[newcol_name] = df[from_col]

    pokemons = [
        {
            'index': index,
            'name': row['NAME'],
            'stats': [row[stat] for stat in ['HP', 'ATTACK', 'DEFENSE', 'SPECIAL-ATTACK', 'SPECIAL-DEFENSE', 'SPEED']],
            'type': [t.strip() for t in row['TYPE'].split(',')],
            newcol_name: row[newcol_name],
            from_col: row[from_col]
        } for index, row in df.iterrows()
    ]

    elo_changes = {poke['index']: 1500 for poke in pokemons}

    for i, poke1 in enumerate(pokemons):
        for j in range(i + 1, len(pokemons)):
            poke2 = pokemons[j]
            new_elo1, new_elo2 = simulator.simulate_round(poke1, poke2, move_adv_df, from_col,
                                                          elo_changes[poke1['index']], elo_changes[poke2['index']])
            elo_changes[poke1['index']] = new_elo1
            elo_changes[poke2['index']] = new_elo2

    for poke in pokemons:
        index = poke['index']
        df.loc[index, newcol_name] = elo_changes[index]

    return df

In [29]:
poke_df = handle_elo(poke_df, 'STATS ELO', 'TOTAL')
poke_df.sample(n=10)

,NAME,TYPE,HP,ATTACK,DEFENSE,SPECIAL-ATTACK,SPECIAL-DEFENSE,SPEED,TOTAL,MOVESET,PRIMARY JOB,SECONDARY JOB,STATS ELO
783,kommo-o,"dragon, fighting",75,110,125,100,105,85,600,"mega-punch, fire-punch, ice-punch, thunder-pun...",Jack-Of-All-Trades,"PSEUDO-HAZER, STALLER, TOXI-SHUFFLERS, PARA-SH...",1929
338,barboach,"water, ground",50,48,43,46,41,60,288,"headbutt, take-down, thrash, double-edge, wate...",Jack-Of-All-Trades,"STALLER, TOXI-SHUFFLERS",1018
1031,giratina-origin,"ghost, dragon",150,120,100,120,100,90,680,"cut, fly, headbutt, body-slam, take-down, roar...",Jack-Of-All-Trades,"PSEUDO-HAZER, STALLER, TOXI-SHUFFLERS, PARA-SH...",2196
1059,charizard-mega-y,"fire, flying",78,104,78,159,115,100,634,"fire-punch, thunder-punch, scratch, swords-dan...",Special Sweeper,"PSEUDO-HAZER, STALLER, TOXI-SHUFFLERS, PARA-SH...",2274
30,nidoqueen,"poison, ground",90,92,87,75,85,76,505,"mega-punch, pay-day, fire-punch, ice-punch, th...",Jack-Of-All-Trades,"PSEUDO-HAZER, STALLER, TOXI-SHUFFLERS, PARA-SH...",1369
644,landorus-incarnate,"ground, flying",89,125,90,115,80,101,600,"swords-dance, fly, body-slam, take-down, leer,...",Mixed Sweeper,"STALLER, TOXI-SHUFFLERS, PIVOTS",2124
40,zubat,"poison, flying",40,45,35,30,40,55,245,"razor-wind, gust, wing-attack, whirlwind, fly,...",Physical Sweeper,"HAZER, PSEUDO-HAZER, STALLER, TOXI-SHUFFLERS, ...",854
732,toucannon,"normal, flying",80,120,75,75,75,60,485,"swords-dance, fly, fury-attack, growl, superso...",Physical Sweeper,"STALLER, TOXI-SHUFFLERS, PIVOTS",1469
136,porygon,normal,65,60,70,85,75,40,395,"headbutt, tackle, take-down, double-edge, ice-...",Jack-Of-All-Trades,"STALLER, TOXI-SHUFFLERS",1028
110,rhyhorn,"ground, rock",80,85,95,30,30,25,345,"swords-dance, stomp, sand-attack, headbutt, ho...",Physical Tank,"PSEUDO-HAZER, STALLER, TOXI-SHUFFLERS, PARA-SH...",1004


### Handling ELO for Primary Jobs

Previously we calculated the elo solely based off their stats for all pokemons, but this time they will be sorted into their specific primary/secondary jobs and then within those groups they will get their elo where they only compete against pokemon within their own jobs

In [30]:
def handle_jobs_elo(poke_df, jobs_array, job_col_name, new_col_name, is_primary):
    poke_primary_elo = {name: 0 for name in poke_df['NAME']}

    for job in jobs_array:
        df = poke_df[poke_df[job_col_name] == job] if is_primary else poke_df[poke_df[job_col_name].str.contains(job)]
        df = handle_elo(df, new_col_name, 'TOTAL')
        for _, row in df.iterrows():
            poke_primary_elo[row['NAME']] += row[new_col_name]

    poke_df[new_col_name] = poke_primary_elo.values()
    return poke_df

In [31]:
primary_jobs = [job for job in job_determine.benchmarks.keys()]
secondary_jobs = [job.upper() for job in job_determine.move_roles.keys()]

poke_df = handle_jobs_elo(poke_df.fillna(' '), primary_jobs, 'PRIMARY JOB', 'PRIMARY JOB ELO', True)
poke_df = handle_jobs_elo(poke_df.fillna(' '), secondary_jobs, 'SECONDARY JOB', 'SECONDARY JOB ELO', False)

poke_df.sample(n=5)

,NAME,TYPE,HP,ATTACK,DEFENSE,SPECIAL-ATTACK,SPECIAL-DEFENSE,SPEED,TOTAL,MOVESET,PRIMARY JOB,SECONDARY JOB,STATS ELO,PRIMARY JOB ELO,SECONDARY JOB ELO
1123,pikachu-alola-cap,electric,35,55,40,50,50,90,320,"mega-punch, pay-day, thunder-punch, slam, mega...",Physical Sweeper,"STALLER, TOXI-SHUFFLERS",1532,1430,1533
2,venusaur,"grass, poison",80,82,83,100,100,80,525,"swords-dance, cut, bind, vine-whip, headbutt, ...",Jack-Of-All-Trades,"PSEUDO-HAZER, STALLER, TOXI-SHUFFLERS, PARA-SH...",1441,1528,2846
466,magmortar,fire,75,95,67,125,95,83,540,"mega-punch, fire-punch, thunder-punch, mega-ki...",Jack-Of-All-Trades,"STALLER, TOXI-SHUFFLERS",1690,1765,1709
274,shiftry,"grass, dark",90,100,60,90,60,80,480,"pound, swords-dance, cut, whirlwind, mega-kick...",Jack-Of-All-Trades,"PSEUDO-HAZER, STALLER, TOXI-SHUFFLERS, PARA-SH...",1486,1570,2965
675,furfrou,normal,75,80,60,65,90,102,472,"sand-attack, headbutt, tackle, take-down, tail...",Jack-Of-All-Trades,"PSEUDO-HAZER, STALLER, TOXI-SHUFFLERS, PARA-SH...",1953,2095,5307


### Calculating the Actual ELO's

To calculate the actual ELO of the pokemon we simply just weigh the stat elo as 1 and then the primary job elo as 0.5, what happens with the secondary job elo is that it is divided by the amount of secondary jobs they have, once this is done we simply just add together the ELO's such that:

ELO = STAT_ELO + 0.5(PRIMARY_JOB_ELO) + 0.5(SECONDARY_JOB_ELO/N) where N is the amount of secondary jobs

In [34]:
poke_df['ELO'] = round(poke_df['STATS ELO'] 
                       + 0.4 * poke_df['PRIMARY JOB ELO'] 
                       + 0.6 * (poke_df['SECONDARY JOB ELO'] / (poke_df['SECONDARY JOB'].str.count(",") + 1)))

In [35]:
poke_df.to_excel('excel_files/ratings.xlsx')

# General Conclusion

Through running the program multiple times, it seems that Zacian is the strongest pokemon

In general legendaries place much higher than most pokemon which is to be expected, the same applies for mega pokemons

Another key point is the fact that types that have larger Effective/Weakness ratio seem to be rated much higher, the reason being that in the simulation those with better type advantages have a higher chance to win, the same applies for speed, the higher the speed the more likely they will win. Due to this reason, Regieleki is rated very highly because of this.

It should be noted that the ratings will differ if the user changes the multpliers and stats that matters, but by the metrics I have done it, these are the ratings. To change the metrics, simply go into simulate.py and chance up the code.

# The Top 20 Pokemon

In [38]:
poke_df.sort_values(by=['ELO'], ascending=False).head(20)

,NAME,TYPE,HP,ATTACK,DEFENSE,SPECIAL-ATTACK,SPECIAL-DEFENSE,SPEED,TOTAL,MOVESET,PRIMARY JOB,SECONDARY JOB,STATS ELO,PRIMARY JOB ELO,SECONDARY JOB ELO,ELO
1212,zacian-crowned,"fairy, steel",92,150,115,80,115,148,700,"swords-dance, body-slam, take-down, bite, hype...",Physical Sweeper,STALLER,2916,2466,2900,5642.0
887,zacian,fairy,92,120,115,80,115,138,660,"swords-dance, body-slam, take-down, bite, hype...",Physical Sweeper,STALLER,2824,2358,2799,5447.0
893,regieleki,electric,80,100,50,100,50,200,580,"body-slam, take-down, thrash, hyper-beam, thun...",Mixed Sweeper,STALLER,2879,2126,2861,5446.0
1295,miraidon-glide-mode,"electric, dragon",100,85,100,135,115,135,670,"swords-dance, body-slam, take-down, hyper-beam...",Special Sweeper,"STALLER, PIVOTS",2893,2110,5338,5338.0
1294,miraidon-aquatic-mode,"electric, dragon",100,85,100,135,115,135,670,"swords-dance, body-slam, take-down, hyper-beam...",Special Sweeper,"STALLER, PIVOTS",2892,2084,5284,5311.0
1293,miraidon-drive-mode,"electric, dragon",100,85,100,135,115,135,670,"swords-dance, body-slam, take-down, hyper-beam...",Special Sweeper,"STALLER, PIVOTS",2855,2062,5249,5254.0
1292,miraidon-low-power-mode,"electric, dragon",100,85,100,135,115,135,670,"swords-dance, body-slam, take-down, hyper-beam...",Special Sweeper,"STALLER, PIVOTS",2848,2034,5190,5219.0
986,flutter-mane,"ghost, fairy",55,55,55,135,135,135,570,"psybeam, hyper-beam, thunderbolt, thunder-wave...",Jack-Of-All-Trades,STALLER,2593,2660,2600,5217.0
1218,calyrex-shadow,"psychic, ghost",100,85,80,165,100,150,680,"pound, pay-day, stomp, double-kick, tackle, bo...",Special Sweeper,"HAZER, CLERIC, STALLER",2876,2108,7226,5164.0
990,iron-bundle,"ice, water",56,80,114,124,60,136,570,"ice-punch, body-slam, take-down, hydro-pump, i...",Special Sweeper,"STALLER, PIVOTS",2704,2062,5104,5060.0
